# Sampling Validation using MC

We can also validate our SBI model by drawing samples from the posterior using MCMC methods. This allows us to assess how well our model captures the underlying distribution of parameters given the observed data.

We can do this using the ```SBI_Fitter.fit_observation_using_sampler``` method, which lets us choose from a few nested sampling or MCMC samplers. Here, we'll use the `dynesty` sampler to draw samples from the posterior.

First we'll load a trained model and choose a mock observation to fit.

```python

In [ ]:
from synference import SBI_Fitter

fitter = SBI_Fitter.load_saved_model("test/test_test_1_posterior.pkl")

First we can recreate the simulator from the information stored in the grid. We will use the simulator in the log-likelihood calculation during sampling.


In [ ]:
fitter.recreate_simulator_from_grid();

Then we can proceed to fit the observation using the sampler.

The code will attempt to remove parameters which don't affect the fit (e.g. supplemental parameters) before fitting, but you can also specify to remove specific parameters using the `remove_params` argument.

Let's choose an observation from the validation set and fit it using the dynesty sampler.


In [ ]:
index = 20
sample = fitter._X_test[index]
true_params = fitter._y_test[index]

print("Sample:", sample)

We'll just recreate our prior for the model.

In [ ]:
fitter.create_priors(set_self=True)

No we can run the sampler to fit the observation. This is too computationally expensive to run here, so we'll just show the code you would use.

```python
result = fitter.fit_observation_using_sampler(
    observation=sample,
    sampler="dynesty",
    sampler_kwargs={"bound": "multi", "sample": "rwalk", "run_kwargs": {"n_effective": 2000}},
    plot_name=f"example_{index}",
    min_flux_pc_error=0.05,
)
samples = result["samples"]
log_l = result["logl"]
log_w = result["logwt"]

```

We can sample using our trained SBI model as well for comparison.

In [ ]:
predicted_params = fitter.sample_posterior(sample, num_samples=1000)